In [1]:
import pandas as pd
import os
import json
import pickle
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from fuzzywuzzy import process
from sklearn.decomposition import TruncatedSVD



In [2]:
# Ouverture fichier ratings
def read_ratings(ratings_csv: str, data_dir: str = "/app/raw") -> pd.DataFrame:
    """
    Lit le fichier CSV contenant les évaluations des films.

    :param ratings_csv: Nom du fichier CSV contenant les évaluations.
    :param data_dir: Répertoire où se trouve le fichier CSV.
    :return: DataFrame contenant les évaluations.
    """
    data = pd.read_csv(os.path.join(data_dir, ratings_csv))
    print("Dataset ratings chargé")
    return data

# Ouverture fichier movies
def read_movies(movies_csv: str, data_dir: str = "/app/raw") -> pd.DataFrame:
    """
    Lit le fichier CSV contenant les informations sur les films.

    :param movies_csv: Nom du fichier CSV contenant les informations sur les films.
    :param data_dir: Répertoire où se trouve le fichier CSV.
    :return: DataFrame contenant les informations sur les films.
    """
    df = pd.read_csv(os.path.join(data_dir, movies_csv))
    print("Dataset movies chargé")
    return df

# Ouverture fichier links
def read_links(links_csv: str, data_dir: str = "/app/raw") -> pd.DataFrame:
    """
    Lit le fichier CSV contenant les informations sur les liens des affiches scrappés.

    :param links_csv: Nom du fichier CSV contenant les liens des affiches.
    :param data_dir: Répertoire où se trouve le fichier CSV.
    :return: DataFrame contenant movieId et lien vers les affiches.
    """
    df = pd.read_csv(os.path.join(data_dir, links_csv))
    df = df[['movieId', 'cover_link']]
    print("Dataset links chargé")
    return df

# Chargement du dernier modèle
def load_model(directory = "/app/model") :
    """Charge le modèle à partir d'un répertoire."""
    # Vérifier si le répertoire existe
    if not os.path.exists(directory):
        raise FileNotFoundError(f"Le répertoire {directory} n'existe pas.")
    # Charger le modèle
    filepath = os.path.join(directory, 'model_knn.pkl')
    with open(filepath, 'rb') as file:
        model = pickle.load(file)
        print(f'Modèle chargé depuis {filepath}')
    return model

# Creation de notre matrice creuse
def create_X(df):
    """
    Génère une matrice creuse avec quatre dictionnaires de mappage
    - user_mapper: mappe l'ID utilisateur à l'index utilisateur
    - movie_mapper: mappe l'ID du film à l'index du film
    - user_inv_mapper: mappe l'index utilisateur à l'ID utilisateur
    - movie_inv_mapper: mappe l'index du film à l'ID du film
    Args:
        df: pandas dataframe contenant 3 colonnes (userId, movieId, rating)
    Returns:
        X: sparse matrix
        user_mapper: dict that maps user id's to user indices
        user_inv_mapper: dict that maps user indices to user id's
        movie_mapper: dict that maps movie id's to movie indices
        movie_inv_mapper: dict that maps movie indices to movie id's
    """
      # Nombre unique d'utilisateurs et de films
    M = df['userId'].nunique()  # Compte le nombre d'utilisateurs uniques
    N = df['movieId'].nunique()  # Compte le nombre de films uniques
    # Créer un dictionnaire pour mapper les IDs utilisateurs à des indices
    user_mapper = dict(zip(np.unique(df["userId"]), list(range(M))))
    # Créer un dictionnaire pour mapper les IDs de films à des indices
    movie_mapper = dict(zip(np.unique(df["movieId"]), list(range(N))))
    # Créer un dictionnaire inverse pour mapper les indices utilisateurs aux IDs utilisateurs
    user_inv_mapper = dict(zip(list(range(M)), np.unique(df["userId"])))
    # Créer un dictionnaire inverse pour mapper les indices de films aux IDs de films
    movie_inv_mapper = dict(zip(list(range(N)), np.unique(df["movieId"])))
    # Obtenir les indices correspondants pour chaque utilisateur et film dans le DataFrame
    user_index = [user_mapper[i] for i in df['userId']]  # Convertir les IDs utilisateurs en indices
    item_index = [movie_mapper[i] for i in df['movieId']]  # Convertir les IDs de films en indices
    # Créer une matrice creuse en utilisant les évaluations, les indices d'utilisateur et de film
    X = csr_matrix((df["rating"], (user_index, item_index)), shape=(M, N))
    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper

# Predictions si utilisateur connu
def find_similar_movies(movie_id, X, movie_mapper, movie_inv_mapper, model, k=10):
    """
    Trouve les k voisins les plus proches pour un ID de film donné.
    Args:
        movie_id: ID du film d'intérêt
        X: matrice d'utilité utilisateur-article (matrice creuse)
        k: nombre de films similaires à récupérer
        metric: métrique de distance pour les calculs kNN
    Output: retourne une liste des k ID de films similaires
    """
    # Transposer la matrice X pour que les films soient en lignes et les utilisateurs en colonnes
    X = X.T
    neighbour_ids = []  # Liste pour stocker les ID des films similaires
    # Obtenir l'index du film à partir du mapper
    movie_ind = movie_mapper[movie_id]
    # Extraire le vecteur correspondant au film spécifié
    movie_vec = X[movie_ind]
    # Vérifier si movie_vec est un tableau NumPy et le remodeler en 2D si nécessaire
    if isinstance(movie_vec, (np.ndarray)):
        movie_vec = movie_vec.reshape(1, -1)  # Reshape pour avoir une forme (1, n_features)
    # Chargement du modèle
    kNN = model

    kNN.fit(X)
    # Trouver les k+1 voisins les plus proches (y compris le film d'intérêt)
    neighbour = kNN.kneighbors(movie_vec, return_distance=False)
    # Collecter les ID des films parmi les voisins trouvés
    for i in range(0, k):  # Boucler jusqu'à k pour obtenir seulement les films similaires
        n = neighbour.item(i)  # Obtenir l'index du voisin
        neighbour_ids.append(movie_inv_mapper[n])  # Mapper l'index à l'ID du film
    neighbour_ids.pop(0)  # Retirer le premier élément qui est l'ID du film original
    return neighbour_ids  # Retourner la liste des ID de films similaires

# Fonction recommendation si utilisateur inconnu
def get_content_based_recommendations(title, n_recommendations=10):
    idx = movie_idx[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:(n_recommendations+1)]
    similar_movies = [i[0] for i in sim_scores]
    return similar_movies

# Validation de l'utilisateur
def validate_userId(userId):
    # Vérifier si userId est dans la plage valide
    if userId < 1 or userId > 138493:
        return "Le numéro d'utilisateur doit être compris entre 1 et 138493."
    return None

# Recherche un titre proche de la requete
def movie_finder(title):
    all_titles = movies['title'].tolist()
    closest_match = process.extractOne(title,all_titles)
    return closest_match[0]

In [3]:
print("DEBUT DES CHARGEMENTS")
ratings = read_ratings('/home/antoine/Ml_Ops_Movies_Reco/app/shared_volume/raw/ratings.csv')
movies = read_movies('/home/antoine/Ml_Ops_Movies_Reco/app/shared_volume/raw/movies.csv')
links = read_links('/home/antoine/Ml_Ops_Movies_Reco/app/shared_volume/raw/links2.csv')
model_knn = load_model('/home/antoine/Ml_Ops_Movies_Reco/app/shared_volume/model')
X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_X(ratings)
svd = TruncatedSVD(n_components=20, n_iter=10)
Q = svd.fit_transform(X.T)
genres = set(g for G in movies['genres'] for g in G)
for g in genres:
    movies[g] = movies.genres.transform(lambda x: int(g in x))
movie_genres = movies.drop(columns=['movieId', 'title','genres'])
movie_idx = dict(zip(movies['title'], list(movies.index)))
cosine_sim = cosine_similarity(movie_genres, movie_genres)
print(f"Dimensions of our genres cosine similarity matrix: {cosine_sim.shape}")
movie_titles = dict(zip(movies['movieId'], movies['title']))
movie_covers = dict(zip(links['movieId'], links['cover_link']))
print("FIN DES CHARGEMENTS")

DEBUT DES CHARGEMENTS
Dataset ratings chargé
Dataset movies chargé
Dataset links chargé
Modèle chargé depuis /home/antoine/Ml_Ops_Movies_Reco/app/shared_volume/model/model_knn.pkl
Dimensions of our genres cosine similarity matrix: (27278, 27278)
FIN DES CHARGEMENTS


In [12]:
movie_title = movie_finder('pokemon')
print(movie_title)

Pokemon 4 Ever (a.k.a. PokÃ©mon 4: The Movie) (2002)


In [8]:
user_id = 10

In [9]:
movie_id = int(movies['movieId'][movies['title'] == movie_title].iloc[0])
print(movie_id)

73


In [13]:
similar_movies = find_similar_movies(movie_id, Q.T, movie_mapper, movie_inv_mapper, model = model_knn, k=11)
print(similar_movies)

[986, 1271, 1416, 1873, 1411, 531, 534, 1087, 1678, 1727]


In [18]:
movie_title = movie_titles[movie_id]
movie_cover = movie_covers[movie_id]
# Créer un dictionnaire pour stocker les titres et les couvertures des films recommandés
result = {
    "user_choice": {"title": movie_title, "cover": movie_cover},
    "recommendations": [{"title": movie_titles[i], "cover": movie_covers[i]} for i in similar_movies]}

print(result)

{'user_choice': {'title': 'MisÃ©rables, Les (1995)', 'cover': 'https://m.media-amazon.com/images/M/MV5BNmYyZjA0YmEtNGU5Ni00YzVjLWJkMTAtZmU0NGI2Y2Y3ODQwXkEyXkFqcGc@._V1_QL75_UY281_CR8,0,190,281_.jpg'}, 'recommendations': [{'title': 'Fly Away Home (1996)', 'cover': 'https://m.media-amazon.com/images/M/MV5BYjQyMzFhYTItODYxYy00MDNmLTgyOGEtNGRmZTg5YzhlNjEyXkEyXkFqcGdeQXVyMjUzOTY1NTc@._V1_QL75_UX190_CR0,1,190,281_.jpg'}, {'title': 'Fried Green Tomatoes (1991)', 'cover': 'https://m.media-amazon.com/images/M/MV5BMzBhMzRkNGItNDFmMy00NjViLWFiYWYtMzAxMzI4YTZmZjRlXkEyXkFqcGc@._V1_QL75_UX190_CR0,1,190,281_.jpg'}, {'title': 'Evita (1996)', 'cover': 'https://m.media-amazon.com/images/M/MV5BODNjMWViYzUtMzk4ZC00NjU4LTlmNjItNzc1ZjRiZGE3Y2E1XkEyXkFqcGc@._V1_QL75_UX190_CR0,1,190,281_.jpg'}, {'title': 'MisÃ©rables, Les (1998)', 'cover': 'https://m.media-amazon.com/images/M/MV5BNjg0NTRjNmUtYzdjYi00YTQ5LTgxNGItMTRhNWYwYzBkOTliXkEyXkFqcGdeQXVyNTAyODkwOQ@@._V1_QL75_UX190_CR0,0,190,281_.jpg'}, {'title': 'Hamlet

In [19]:
user_choise = result['user_choice']
print(user_choise)

{'title': 'MisÃ©rables, Les (1995)', 'cover': 'https://m.media-amazon.com/images/M/MV5BNmYyZjA0YmEtNGU5Ni00YzVjLWJkMTAtZmU0NGI2Y2Y3ODQwXkEyXkFqcGc@._V1_QL75_UY281_CR8,0,190,281_.jpg'}


In [20]:
print(user_choise['title'])

MisÃ©rables, Les (1995)


In [22]:
recommended_movies = result['recommendations']
print(recommended_movies)

[{'title': 'Fly Away Home (1996)', 'cover': 'https://m.media-amazon.com/images/M/MV5BYjQyMzFhYTItODYxYy00MDNmLTgyOGEtNGRmZTg5YzhlNjEyXkEyXkFqcGdeQXVyMjUzOTY1NTc@._V1_QL75_UX190_CR0,1,190,281_.jpg'}, {'title': 'Fried Green Tomatoes (1991)', 'cover': 'https://m.media-amazon.com/images/M/MV5BMzBhMzRkNGItNDFmMy00NjViLWFiYWYtMzAxMzI4YTZmZjRlXkEyXkFqcGc@._V1_QL75_UX190_CR0,1,190,281_.jpg'}, {'title': 'Evita (1996)', 'cover': 'https://m.media-amazon.com/images/M/MV5BODNjMWViYzUtMzk4ZC00NjU4LTlmNjItNzc1ZjRiZGE3Y2E1XkEyXkFqcGc@._V1_QL75_UX190_CR0,1,190,281_.jpg'}, {'title': 'MisÃ©rables, Les (1998)', 'cover': 'https://m.media-amazon.com/images/M/MV5BNjg0NTRjNmUtYzdjYi00YTQ5LTgxNGItMTRhNWYwYzBkOTliXkEyXkFqcGdeQXVyNTAyODkwOQ@@._V1_QL75_UX190_CR0,0,190,281_.jpg'}, {'title': 'Hamlet (1996)', 'cover': 'https://m.media-amazon.com/images/M/MV5BNGQ0OGUyMTctNzk4ZS00YmY5LTlmNWYtMWIyZWYwZjRmOWIzXkEyXkFqcGdeQXVyMTU3NDU4MDg2._V1_QL75_UY281_CR1,0,190,281_.jpg'}, {'title': 'Secret Garden, The (1993)', 'cover'

In [23]:
for i, movie in enumerate(recommended_movies):
    print(movie['title'])

Fly Away Home (1996)
Fried Green Tomatoes (1991)
Evita (1996)
MisÃ©rables, Les (1998)
Hamlet (1996)
Secret Garden, The (1993)
Shadowlands (1993)
Madame Butterfly (1995)
Joy Luck Club, The (1993)
Horse Whisperer, The (1998)


In [13]:
similar_movies = get_content_based_recommendations(movie_title, n_recommendations=10)

In [14]:
print(similar_movies)

[1765, 1949, 2032, 2209, 3027, 3312, 3663, 3922, 4271, 4424]


In [23]:
movies[movies.index == 3027].head()

,movieId,title,genres,M,l,o,R,I,),W,...,d,F,u,A,t,r,m,g,,v
3027,3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy,0,1,1,0,0,0,0,...,1,1,1,1,1,1,1,0,0,1


In [15]:
movie_id = int(movies['movieId'][movies['title'] == movie_title].iloc[0])

In [16]:
movie_title = movie_titles[movie_id]
movie_cover = movie_covers[movie_id]

In [20]:
print(movie_title)

Pokemon 4 Ever (a.k.a. PokÃ©mon 4: The Movie) (2002)


In [17]:
result = {
        "user_choice": {"title": movie_title, "cover": movie_cover},
        "recommendations": [{"title": movie_titles[i], "cover": movie_covers[i]} for i in similar_movies]
    }

KeyError: 3027

In [19]:
movie_titles[3028]

'Taming of the Shrew, The (1967)'

In [11]:
user_choice = result['user_choice']
print(user_choice)

{'title': 'MisÃ©rables, Les (1995)', 'cover': 'https://m.media-amazon.com/images/M/MV5BNmYyZjA0YmEtNGU5Ni00YzVjLWJkMTAtZmU0NGI2Y2Y3ODQwXkEyXkFqcGc@._V1_QL75_UY281_CR8,0,190,281_.jpg'}


In [29]:

recommended_movies = result['recommendations']
for i, movie in enumerate(recommended_movies):
    print(movie['title'])

Kicking and Screaming (1995)
Bushwhacked (1995)
Village of the Damned (1995)
Ruby in Paradise (1993)
Asfour Stah (1990)
Stupids, The (1996)
Little Lord Fauntleroy (1936)
Crossfire (1947)
Dadetown (1995)
Invitation, The (Zaproszenie) (1986)


In [25]:
print(movie_idx)

{'Toy Story (1995)': 0, 'Jumanji (1995)': 1, 'Grumpier Old Men (1995)': 2, 'Waiting to Exhale (1995)': 3, 'Father of the Bride Part II (1995)': 4, 'Heat (1995)': 5, 'Sabrina (1995)': 6, 'Tom and Huck (1995)': 7, 'Sudden Death (1995)': 8, 'GoldenEye (1995)': 9, 'American President, The (1995)': 10, 'Dracula: Dead and Loving It (1995)': 11, 'Balto (1995)': 12, 'Nixon (1995)': 13, 'Cutthroat Island (1995)': 14, 'Casino (1995)': 15, 'Sense and Sensibility (1995)': 16, 'Four Rooms (1995)': 17, 'Ace Ventura: When Nature Calls (1995)': 18, 'Money Train (1995)': 19, 'Get Shorty (1995)': 20, 'Copycat (1995)': 21, 'Assassins (1995)': 22, 'Powder (1995)': 23, 'Leaving Las Vegas (1995)': 24, 'Othello (1995)': 25, 'Now and Then (1995)': 26, 'Persuasion (1995)': 27, 'City of Lost Children, The (CitÃ© des enfants perdus, La) (1995)': 28, 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)': 29, 'Dangerous Minds (1995)': 30, 'Twelve Monkeys (a.k.a. 12 Monkeys) (1995)': 31, 'Wings of Courage (1995)':

Toy Story (1995)


['Toy Story (1995)',
 'Jumanji (1995)',
 'Grumpier Old Men (1995)',
 'Waiting to Exhale (1995)',
 'Father of the Bride Part II (1995)',
 'Heat (1995)',
 'Sabrina (1995)',
 'Tom and Huck (1995)',
 'Sudden Death (1995)',
 'GoldenEye (1995)',
 'American President, The (1995)',
 'Dracula: Dead and Loving It (1995)',
 'Balto (1995)',
 'Nixon (1995)',
 'Cutthroat Island (1995)',
 'Casino (1995)',
 'Sense and Sensibility (1995)',
 'Four Rooms (1995)',
 'Ace Ventura: When Nature Calls (1995)',
 'Money Train (1995)',
 'Get Shorty (1995)',
 'Copycat (1995)',
 'Assassins (1995)',
 'Powder (1995)',
 'Leaving Las Vegas (1995)',
 'Othello (1995)',
 'Now and Then (1995)',
 'Persuasion (1995)',
 'City of Lost Children, The (CitÃ© des enfants perdus, La) (1995)',
 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 'Dangerous Minds (1995)',
 'Twelve Monkeys (a.k.a. 12 Monkeys) (1995)',
 'Wings of Courage (1995)',
 'Babe (1995)',
 'Carrington (1995)',
 'Dead Man Walking (1995)',
 'Across the Sea of